In [63]:
example = """
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
""".strip()

with open("day16.txt", "r") as f:
    data = f.read()

In [64]:
from re import match

class Parser(object):
    def __init__(self, data: str):
        self.input = data
        self.position = 0

    def reset(self):
        self.position = 0

    def peek(self, pattern: str = None) -> str:
        if pattern is None:
            return self.input[self.position:]

        token = match(pattern, self.input[self.position:])
        if token is None:
            return None

        return token.group(0)
        
    def consume(self, pattern: str) -> str:
        token = self.peek(pattern)
        if token is None:
            return None
        
        self.position += len(token)
        return token

    def gobble(self, pattern: str) -> bool:
        return self.consume(pattern) is not None

    def gobble_newline(self) -> bool:
        return self.gobble(r'\n')

    def gobble_whitespace(self) -> bool:
        return self.gobble(r'\s+')

example_parser = Parser(example)
assert example_parser.peek().startswith("class:")
assert example_parser.consume(r'\w+') == "class"
assert example_parser.gobble(r':\s+')
assert example_parser.consume(r'\d+') == "1"
assert example_parser.gobble(r'-')
assert example_parser.consume(r'\d+') == "3"
example_parser.reset()

## Models
These models describe the entities contained within our input data.

In [65]:
from typing import List, Tuple, Iterator, Dict

In [66]:
class Range(object):
    def __init__(self, minimum: int, maximum: int):
        self.min = minimum
        self.max = maximum

    @staticmethod
    def parse(parser: Parser) -> "Range":
        lower = parser.consume(r'\d+')
        assert lower is not None

        assert parser.gobble(r'-')

        upper = parser.consume(r'\d+')
        assert upper is not None

        return Range(int(lower), int(upper))
        
    def __contains__(self, value: int):
        return self.min <= value <= self.max

    def __str__(self) -> str:
        return f"{self.min}-{self.max}"

    def __repr__(self) -> str:
        return f"Range({self.min}, {self.max})"

In [94]:
from typing import Tuple

class Ticket(object):
    def __init__(self, fields: List[int]):
        self.fields = fields

    @staticmethod
    def parse(parser: Parser) -> "Ticket":
        spec = parser.consume(r"[\d,]+")
        if spec is None:
            return None

        parser.gobble_newline()

        return Ticket(list(map(int, spec.split(','))))

    def validate(self, valid_fields: List["Field"]) -> Tuple[bool, int]:
        valid, errors = True, 0
        for field in self.fields:
            if all(field not in valid_field for valid_field in valid_fields):
                valid = False
                errors += field

        return (valid, errors)

    def to_dict(self, ticket_field_order: Dict[int, "Field"]) -> Dict[str, int]:
        return {
            ticket_field_order[index].name: value
            for index, value in enumerate(self.fields)
        }

In [95]:
class Field(object):
    def __init__(self, name: str, ranges: List[Range]):
        self.name = name
        self.ranges = ranges

    @staticmethod
    def parse(parser: Parser) -> "Field":
        name = parser.consume(r'\w[\w ]*')
        if name is None:
            return None

        assert parser.gobble(r': ')
        ranges = []

        while parser.peek(r'\d'):
            new_range = Range.parse(parser)
            if new_range is None:
                break

            ranges.append(new_range)
            parser.gobble(r' +or +')

        parser.gobble_newline()

        return Field(name, ranges)

    def __contains__(self, value: int) -> bool:
        return any(value in valid_range for valid_range in self.ranges)

    def __str__(self) -> str:
        return f'{self.name}: {" or ".join(map(str, self.ranges))}'

    def __repr__(self) -> str:
        return f"Field('{self.name}', [{self.ranges}])"

In [99]:
from typing import Iterator

def mul(values: Iterator[float]) -> float:
    product = 1
    for value in values:
        product *= value

    return product

In [102]:
class Problem(object):
    def __init__(self, fields: List[Field], my_ticket: Ticket, nearby_tickets: List[Ticket]):
        self.fields = fields
        self.my_ticket = my_ticket
        self.nearby_tickets = nearby_tickets

    @staticmethod
    def parse(parser: Parser) -> "Problem":
        fields = []
        while True:
            new_field = Field.parse(parser)
            if new_field is None:
                break

            fields.append(new_field)

        assert parser.gobble_newline()
        assert parser.gobble(r"your ticket:\n")
        my_ticket = Ticket.parse(parser)
        assert my_ticket is not None

        assert parser.gobble_newline()
        assert parser.gobble(r"nearby tickets:\n")
        nearby_tickets = []
        while True:
            ticket = Ticket.parse(parser)
            if ticket is None:
                break

            nearby_tickets.append(ticket)

        return Problem(fields, my_ticket, nearby_tickets)

    def get_scanning_error_rate(self):
        return sum(ticket.validate(self.fields)[1] for ticket in self.nearby_tickets)

    def get_valid_tickets(self) -> Iterator[Ticket]:
        for ticket in self.nearby_tickets:
            if ticket.validate(self.fields)[0]:
                yield ticket

    def get_ticket_field_order(self) -> Dict[int, Field]:
        valid_tickets = [
            self.my_ticket,
            *self.get_valid_tickets()
        ]

        # We first get the set of values used by valid tickets in each field index
        field_values = {
            index: set(ticket.fields[index] for ticket in valid_tickets)
            for index in range(0, len(self.my_ticket.fields))
        }

        field_candidates = sorted([
            (field, set(index for index, values in field_values.items() if all(value in field for value in values)))
            for field in self.fields
        ], key=lambda entry: len(entry[1]))
        #print({ entry[0].name: entry[1] for entry in field_candidates })

        assert all(entry[1] for entry in field_candidates)

        ordering = {}

        while field_candidates:
            candidate = next(iter(field_candidates))

            assert candidate[1]

            index = next(iter(candidate[1]))
            ordering[index] = candidate[0]

            # Update our list of candidates to be assigned
            field_candidates = sorted([
                (entry[0], entry[1] - set([index]))
                for entry in field_candidates
                if entry != candidate
            ], key=lambda entry: len(entry[1]))

            # print({ entry[0].name: entry[1] for entry in field_candidates })

        return ordering

        
example_parser.reset()
example_problem = Problem.parse(example_parser)
assert set(field.name for field in example_problem.fields) == set(["class", "row", "seat"])
assert example_problem.my_ticket.fields == [7, 1, 14]

assert example_problem.get_scanning_error_rate() == 71

# Example for second part
example2 = """
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
""".strip()
example_parser2 = Parser(example2)
example_problem2 = Problem.parse(example_parser2)
assert set(field.name for field in example_problem2.fields) == set(["class", "row", "seat"])
assert example_problem2.my_ticket.fields == [11, 12, 13]

test_field_order = example_problem2.get_ticket_field_order()
assert set(field.name for field in test_field_order.values()) == set(["class", "row", "seat"])
test_ticket = example_problem2.my_ticket.to_dict(test_field_order)
assert test_ticket == {
    "row": 11,
    "class": 12,
    "seat": 13
}


real_parser = Parser(data)
real_problem = Problem.parse(real_parser)
print(f"Scanning Error Rate (part 1): {real_problem.get_scanning_error_rate()}")

real_ticket = real_problem.my_ticket.to_dict(real_problem.get_ticket_field_order())
print(f"Your Ticket (part 2): {mul(value for key, value in real_ticket.items() if key.startswith('departure'))}")

Scanning Error Rate (part 1): 24021
Your Ticket (part 2): 1289178686687
